In [1]:
from transformers import pipeline
from collections import defaultdict

# Replace this with your own checkpoint
model_checkpoint = "../models/epoch_30"
token_classifier = pipeline(
    "ner", model=model_checkpoint, aggregation_strategy="simple"
)

2023-10-24 13:59:34.865104: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-24 13:59:34.891097: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 13:59:35.323570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def preprocess_description(description):
    single_description = description.strip()
    new_description = []
    last_special = -1
    for idx, letter in enumerate(single_description):
        if not (('a' <= letter and letter <= 'z') or ('A' <= letter and letter <= 'Z') or ('0' <= letter and letter <= '9') or letter == ' '):
            pretext = single_description[last_special + 1:idx].strip()
            if pretext != '' and pretext != ' ':
                new_description.append(pretext)
            new_description.append(letter.strip())
            last_special = idx
        if idx == len(single_description) - 1:
            new_description.append(
                single_description[last_special + 1:idx + 1].strip())
    return " ".join(new_description)

In [4]:
import json

# Replace this description
description = '''
Includes ivory linen cushions and features a natural ash frame with natural-toned French cane sides and back
'''

high_score_ans = defaultdict(set)
bullet_points = description.split("\n")
for bullet_point in bullet_points:
    bullet_point = preprocess_description(bullet_point)

    if bullet_point != "":
        print(bullet_point)

        results = token_classifier(bullet_point)
        for res in results:
            if res['word'].lower().strip() in ['durable', 'strong', 'heavy-duty', 'heavy duty', 'stability', 'versatile']:
                continue
            group = res['entity_group']
            if res['score'] >= 0.9:
                high_score_ans[group].add(res['word'].lower().strip())
                
new_high_score_ans = defaultdict(list)
for key_dict in high_score_ans.keys():
    new_high_score_ans[key_dict] = list(high_score_ans[key_dict])

print("-"*100)            
print(json.dumps(new_high_score_ans, sort_keys=True, indent=4))                

Includes ivory linen cushions and features a natural ash frame with natural - toned French cane sides and back
----------------------------------------------------------------------------------------------------
{
    "MAT": [
        "natural ash",
        "french cane",
        "linen",
        "ivory"
    ],
    "PROPERTY": [
        "natural - toned"
    ]
}
